# ML Pipeline Preparation
This is a simple machine learning script that looks at tweets during natural disasters and attempts to map them to 36 different categories. The result of this can then be used to help emergency services give aid to those who truly require it.

### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [9]:
# import libraries
import pandas as pd
import sqlalchemy
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [30]:
# load data from database
engine = sqlalchemy.create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('Message', engine)
X = df['message']
Y = df.iloc[:,5:] #Skip genre and add later after one hot encoding
Y_dummies = pd.get_dummies(df[['genre']].astype(str))
Y = Y.merge(Y_dummies, how='inner',left_index=True,right_index=True, validate='1:1')
Y['related'] = Y['related'].apply(lambda x: 1 if x==2 else x) #Change 2s into 1s so this is binary
print(Y.head())
print(X.head())

   related  request  offer  aid_related  medical_help  medical_products  \
0        1        0      0            0             0                 0   
1        1        0      0            1             0                 0   
2        1        0      0            0             0                 0   
3        1        1      0            1             0                 1   
4        1        0      0            0             0                 0   

   search_and_rescue  security  military  child_alone  ...  floods  storm  \
0                  0         0         0            0  ...       0      0   
1                  0         0         0            0  ...       0      1   
2                  0         0         0            0  ...       0      0   
3                  0         0         0            0  ...       0      0   
4                  0         0         0            0  ...       0      0   

   fire  earthquake  cold  other_weather  direct_report  genre_direct  \
0     0      

In [41]:
df.iloc[:,5:].sum().values

array([20282,  4474,   118, 10860,  2084,  1313,   724,   471,   860,
           0,  1672,  2923,  2314,   405,   604,   298,   875,  1194,
        3446,  1705,  1201,  1333,   532,   159,   283,   120,   309,
        1151,  7297,  2155,  2443,   282,  2455,   530,  1376,  5075],
      dtype=int64)

### 2. A simple tokenization function to process text data

In [11]:
def tokenize(text):
    """Takes a piece of text and does the following:
        - Removes stop words
        - Lemmatizes words
        - Removes punctuation
        - Lower cases all words
        - Tokenizes words

    Keyword arguments:
    text - string containing piece of text
    
    Returns:
    tokens - array containing tokens associated with that text
    """
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

### 3. A machine learning pipeline
This machine pipeline takes in the `message` column as input and output classification results on the other 36 categories in the dataset.

In [12]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(KNeighborsClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=226)
pipeline.fit(X_train, y_train)
pipeline.predict(X[-2:]) 

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]], dtype=int64)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [15]:
y_pred = pipeline.predict(X_test)

In [16]:
for i in range(y_pred.shape[1]):
    col_name = y_test.columns[i]
    y_test_i = y_test.iloc[:,i]
    y_pred_i = [item[i] for item in y_pred]
    print(col_name + ":")
    print(classification_report(y_test_i, y_pred_i))

related:
              precision    recall  f1-score   support

           0       0.58      0.32      0.41      1489
           1       0.82      0.93      0.87      5008
           2       0.39      0.30      0.34        57

    accuracy                           0.78      6554
   macro avg       0.59      0.52      0.54      6554
weighted avg       0.76      0.78      0.76      6554

request:
              precision    recall  f1-score   support

           0       0.84      0.99      0.91      5414
           1       0.72      0.08      0.14      1140

    accuracy                           0.83      6554
   macro avg       0.78      0.53      0.52      6554
weighted avg       0.82      0.83      0.77      6554

offer:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6524
           1       0.00      0.00      0.00        30

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      

c:\users\mnasra\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6233
           1       0.00      0.00      0.00       321

    accuracy                           0.95      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       0.90      0.95      0.93      6554

buildings:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6230
           1       0.67      0.01      0.02       324

    accuracy                           0.95      6554
   macro avg       0.81      0.51      0.50      6554
weighted avg       0.94      0.95      0.93      6554

electricity:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6402
           1       0.00      0.00      0.00       152

    accuracy                           0.98      6554
   macro avg       0.49      0.50      0.49      6554
weighted avg       0.95      0.98      0.97      6

### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x0000025119548670>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=KNeighborsClassifier(algorithm='auto',
                                                        leaf_size=30,
                                                        metric='minkowski',
                                                        metric_params=None,
                                                        n_jobs=None, n

In [18]:
parameters = {
    
    'vect__ngram_range':[(1,1),(1,2)],
    'tfidf__use_idf': (True, False), 
    'clf__estimator__p':[1,2]
}

cv = GridSearchCV(pipeline, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

In [20]:
cv.cv_results_

{'mean_fit_time': array([10.5552403 , 10.89437551, 10.21127052, 10.67087703, 10.2406476 ,
        10.91435084, 10.37285361, 11.09889569]),
 'std_fit_time': array([0.43297501, 0.31727121, 0.11747734, 0.11166314, 0.02991926,
        0.0917454 , 0.11987317, 0.33392826]),
 'mean_score_time': array([76.23952723, 91.4764329 , 74.57840424, 89.91234956, 51.03684735,
        52.41938567, 51.45226359, 53.36828218]),
 'std_score_time': array([3.16332029, 2.06138028, 2.70714012, 2.5388852 , 0.45217798,
        1.13271211, 0.41024321, 1.04718865]),
 'param_clf__estimator__p': masked_array(data=[1, 1, 1, 1, 2, 2, 2, 2],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_tfidf__use_idf': masked_array(data=[True, True, False, False, True, True, False, False],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_vect__ngram_range': mas

In [21]:
print(cv.cv_results_['rank_test_score'])
print(cv.cv_results_['rank_test_score'].tolist().index(1))

[6 8 5 7 4 3 1 2]
6


In [22]:
#Best set of parameters was at index 6 so lets look at that
cv.cv_results_['params'][6]

{'clf__estimator__p': 2, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 1)}

In [23]:
y_pred = cv.predict(X_test)
for i in range(y_pred.shape[1]):
    col_name = y_test.columns[i]
    y_test_i = y_test.iloc[:,i]
    y_pred_i = [item[i] for item in y_pred]
    print(col_name + ":")
    print(classification_report(y_test_i, y_pred_i))

related:
              precision    recall  f1-score   support

           0       0.61      0.37      0.46      1489
           1       0.83      0.92      0.87      5008
           2       0.34      0.33      0.34        57

    accuracy                           0.79      6554
   macro avg       0.59      0.54      0.56      6554
weighted avg       0.77      0.79      0.77      6554

request:
              precision    recall  f1-score   support

           0       0.86      0.99      0.92      5414
           1       0.79      0.24      0.37      1140

    accuracy                           0.86      6554
   macro avg       0.83      0.62      0.65      6554
weighted avg       0.85      0.86      0.82      6554

offer:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6524
           1       0.00      0.00      0.00        30

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      

c:\users\mnasra\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6230
           1       0.67      0.02      0.04       324

    accuracy                           0.95      6554
   macro avg       0.81      0.51      0.51      6554
weighted avg       0.94      0.95      0.93      6554

electricity:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6402
           1       0.00      0.00      0.00       152

    accuracy                           0.98      6554
   macro avg       0.49      0.50      0.49      6554
weighted avg       0.95      0.98      0.97      6554

tools:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6508
           1       0.00      0.00      0.00        46

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [24]:
import pickle
with open('models/modelv1.pkl', 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.